## Setup Inicial

1. Carregar as bibliotecas
2. Registrar as configurações
3. Carregar os dados
4. Realizar uma análise exploratória

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

# Configurações
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

## Importando os dados

> Selecionando as colunas de interesse, me baseando no dicionário de dados

In [2]:
# Selecionar as colunas de interesse
cols = [
    'NU_INSCRICAO',
    'TP_PRESENCA_CN',
    'TP_PRESENCA_CH',
    'TP_PRESENCA_LC',
    'TP_PRESENCA_MT',
    'NU_NOTA_CN',
    'NU_NOTA_CH',
    'NU_NOTA_LC',
    'NU_NOTA_MT',
    'NU_NOTA_REDACAO',
    'Q006',  # Pergunta sobre a renda familiar
    'Q025'   # Pergunta sobre o acesso à internet em casa
]

> Por ser um arquivo com quase 3 milhões de linhas, vamos carregar os dados em chunks

In [3]:
# Carregando os dados
df = pd.read_csv('../data/MICRODADOS_ENEM_2023.csv/MICRODADOS_ENEM_2023.csv', sep=';', encoding='latin-1')

In [4]:
file_path = '../data/MICRODADOS_ENEM_2023.csv/MICRODADOS_ENEM_2023.csv'

# Carregar os dados em chunks
chunks = pd.read_csv(
    file_path,
    sep=';',
    encoding='latin-1',
    usecols=cols,
    chunksize=100000,
)
df = pd.concat(chunks)

In [5]:
# Exibir as primeiras linhas do DataFrame
display(df.head())

,NU_INSCRICAO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,Q006,Q025
0,210059085136,0,0,0,0,NaN,NaN,NaN,NaN,NaN,F,B
1,210059527735,0,0,0,0,NaN,NaN,NaN,NaN,NaN,H,B
2,210061103945,1,1,1,1,502.0,498.9,475.6,363.2,700.0,C,B
3,210060214087,1,1,1,1,459.0,508.5,507.2,466.7,880.0,C,B
4,210059980948,1,1,1,1,402.5,379.2,446.9,338.3,560.0,B,A


In [6]:
# quantidade de linhas x quantidade de colunas
# O resultado é uma tupla com o número de linhas e colunas
df.shape

(3933955, 12)

In [7]:
# Sumário do DataFrame
# Número de valores não nulos, dtype de cada coluna, memory usage
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933955 entries, 0 to 3933954
Data columns (total 12 columns):
 #   Column           Dtype  
---  ------           -----  
 0   NU_INSCRICAO     int64  
 1   TP_PRESENCA_CN   int64  
 2   TP_PRESENCA_CH   int64  
 3   TP_PRESENCA_LC   int64  
 4   TP_PRESENCA_MT   int64  
 5   NU_NOTA_CN       float64
 6   NU_NOTA_CH       float64
 7   NU_NOTA_LC       float64
 8   NU_NOTA_MT       float64
 9   NU_NOTA_REDACAO  float64
 10  Q006             object 
 11  Q025             object 
dtypes: float64(5), int64(5), object(2)
memory usage: 360.2+ MB


> Temos apenas duas opções possíveis para 'Acesso à internet': Sim (A) ou Não (B)

In [8]:
# Ver distribuição de acesso à internet
df['Q025'].value_counts()

Q025
B    3558451
A     375504
Name: count, dtype: int64

> Temos várias opções possíveis para 'Renda Familiar'

In [9]:
# Ver distribuição de renda
df['Q006'].value_counts()

Q006
B    1245271
C     650942
D     437366
E     293994
A     268053
G     261327
F     171344
H     139279
I      85970
J      75179
K      59631
Q      51489
N      41565
O      41218
L      39596
P      38105
M      33626
Name: count, dtype: int64

> Os dados ausentes estão apenas nas provas, provavelmente porque os alunos não fizeram essas provas

In [10]:
# Verificar dados ausentes
df.isna().sum().sort_values(ascending=False)

NU_NOTA_MT         1241528
NU_NOTA_CN         1241528
NU_NOTA_CH         1111312
NU_NOTA_LC         1111312
NU_NOTA_REDACAO    1111312
TP_PRESENCA_CH           0
TP_PRESENCA_CN           0
NU_INSCRICAO             0
TP_PRESENCA_LC           0
TP_PRESENCA_MT           0
Q006                     0
Q025                     0
dtype: int64

In [11]:
# Verificar porcentagem de dados ausentes
(df.isna().sum() / df.shape[0] * 100).round(2).sort_values(ascending=False)

NU_NOTA_MT         31.56
NU_NOTA_CN         31.56
NU_NOTA_CH         28.25
NU_NOTA_LC         28.25
NU_NOTA_REDACAO    28.25
TP_PRESENCA_CH      0.00
TP_PRESENCA_CN      0.00
NU_INSCRICAO        0.00
TP_PRESENCA_LC      0.00
TP_PRESENCA_MT      0.00
Q006                0.00
Q025                0.00
dtype: float64

In [12]:
# Filtrar apenas alunos que fizeram todas as provas
df = df[
    (df['TP_PRESENCA_CN'] == 1) & 
    (df['TP_PRESENCA_CH'] == 1) & 
    (df['TP_PRESENCA_LC'] == 1) & 
    (df['TP_PRESENCA_MT'] == 1)
]

display(df.head())

,NU_INSCRICAO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,Q006,Q025
2,210061103945,1,1,1,1,502.0,498.9,475.6,363.2,700.0,C,B
3,210060214087,1,1,1,1,459.0,508.5,507.2,466.7,880.0,C,B
4,210059980948,1,1,1,1,402.5,379.2,446.9,338.3,560.0,B,A
9,210060801601,1,1,1,1,564.7,630.3,610.4,680.2,600.0,F,B
10,210059085130,1,1,1,1,644.9,620.2,626.9,736.3,860.0,B,B


> Após filtrar os alunos que marcaram presença, não há mais dados ausentes.

In [13]:
# Verificar se ainda existem dados ausentes
df.isna().sum().sort_values(ascending=False)

NU_INSCRICAO       0
TP_PRESENCA_CN     0
TP_PRESENCA_CH     0
TP_PRESENCA_LC     0
TP_PRESENCA_MT     0
NU_NOTA_CN         0
NU_NOTA_CH         0
NU_NOTA_LC         0
NU_NOTA_MT         0
NU_NOTA_REDACAO    0
Q006               0
Q025               0
dtype: int64

> Vamos criar uma nova coluna com a nota média, no intuito de facilitar os gráficos

In [14]:
# Pegando as colunas de provas
notas = df.columns[df.columns.str.contains('NOTA')].tolist()

# Criar uma nota média
df['NU_NOTA_MEDIA'] = df[notas].mean(axis=1)

display(df)

,NU_INSCRICAO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,Q006,Q025,NU_NOTA_MEDIA
2,210061103945,1,1,1,1,502.0,498.9,475.6,363.2,700.0,C,B,507.94
3,210060214087,1,1,1,1,459.0,508.5,507.2,466.7,880.0,C,B,564.28
4,210059980948,1,1,1,1,402.5,379.2,446.9,338.3,560.0,B,A,425.38
9,210060801601,1,1,1,1,564.7,630.3,610.4,680.2,600.0,F,B,617.12
10,210059085130,1,1,1,1,644.9,620.2,626.9,736.3,860.0,B,B,697.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3933946,210061965964,1,1,1,1,568.6,605.2,598.2,496.8,500.0,D,B,553.76
3933948,210061959674,1,1,1,1,476.2,542.9,545.2,530.5,600.0,F,B,538.96
3933950,210061959676,1,1,1,1,566.2,605.6,613.7,547.3,540.0,F,B,574.56
3933951,210061950911,1,1,1,1,377.2,535.6,610.6,644.4,640.0,F,B,561.56


> Vamos remapear as opções de resposta das colunas de perguntas para torná-las mais legíveis.

In [16]:
# Mapear códigos para legendas mais amigáveis
internet_map = {
    'A': 'Não tem internet em casa',
    'B': 'Tem internet em casa'
}

In [17]:
# Mapear códigos para legendas mais amigáveis
renda_map = {
    'A': 'Nenhuma renda',
    'B': 'Até R$ 1.100,00',
    'C': 'De R$ 1.100,01 até R$ 1.650,00.',
    'D': 'De R$ 1.650,01 até R$ 2.200,00.',
    'E': 'De R$ 2.200,01 até R$ 2.750,00.',
    'F': 'De R$ 2.750,01 até R$ 3.300,00.',
    'G': 'De R$ 3.300,01 até R$ 4.400,00.',
    'H': 'De R$ 4.400,01 até R$ 5.500,00.',
    'I': 'De R$ 5.500,01 até R$ 6.600,00.',
    'J': 'De R$ 6.600,01 até R$ 7.700,00.',
    'K': 'De R$ 7.700,01 até R$ 8.800,00.',
    'L': 'De R$ 8.800,01 até R$ 9.900,00.',
    'M': 'De R$ 9.900,01 até R$ 11.000,00.',
    'N': 'De R$ 11.000,01 até R$ 13.200,00.',
    'O': 'De R$ 13.200,01 até R$ 16.500,00.',
    'P': 'De R$ 16.500,01 até R$ 22.000,00.',
    'Q': 'Acima de R$ 22.000,00.'
}

In [18]:
# Aplicar mapeamento no DataFrame
df['INTERNET'] = df['Q025'].map(internet_map)
df['RENDA_FAMILIAR'] = df['Q006'].map(renda_map)

display(df)

,NU_INSCRICAO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,Q006,Q025,NU_NOTA_MEDIA,INTERNET,RENDA_FAMILIAR
2,210061103945,1,1,1,1,502.0,498.9,475.6,363.2,700.0,C,B,507.94,Tem internet em casa,"De R$ 1.100,01 até R$ 1.650,00."
3,210060214087,1,1,1,1,459.0,508.5,507.2,466.7,880.0,C,B,564.28,Tem internet em casa,"De R$ 1.100,01 até R$ 1.650,00."
4,210059980948,1,1,1,1,402.5,379.2,446.9,338.3,560.0,B,A,425.38,Não tem internet em casa,"Até R$ 1.100,00"
9,210060801601,1,1,1,1,564.7,630.3,610.4,680.2,600.0,F,B,617.12,Tem internet em casa,"De R$ 2.750,01 até R$ 3.300,00."
10,210059085130,1,1,1,1,644.9,620.2,626.9,736.3,860.0,B,B,697.66,Tem internet em casa,"Até R$ 1.100,00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3933946,210061965964,1,1,1,1,568.6,605.2,598.2,496.8,500.0,D,B,553.76,Tem internet em casa,"De R$ 1.650,01 até R$ 2.200,00."
3933948,210061959674,1,1,1,1,476.2,542.9,545.2,530.5,600.0,F,B,538.96,Tem internet em casa,"De R$ 2.750,01 até R$ 3.300,00."
3933950,210061959676,1,1,1,1,566.2,605.6,613.7,547.3,540.0,F,B,574.56,Tem internet em casa,"De R$ 2.750,01 até R$ 3.300,00."
3933951,210061950911,1,1,1,1,377.2,535.6,610.6,644.4,640.0,F,B,561.56,Tem internet em casa,"De R$ 2.750,01 até R$ 3.300,00."


> Simplicamos ainda mais a renda, dessa vez para categorias mais amplas

In [19]:
# Simplificar renda para categorias mais amplas
def simplify_income(income_code):
    if income_code in ['A', 'B', 'C', 'D']:
        return 'Baixa renda'
    elif income_code in ['E', 'F', 'G', 'H']:
        return 'Média renda'
    else:
        return 'Alta renda'

df['RENDA_SIMPLIFICADA'] = df['Q006'].apply(simplify_income)

display(df.head())

,NU_INSCRICAO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,Q006,Q025,NU_NOTA_MEDIA,INTERNET,RENDA_FAMILIAR,RENDA_SIMPLIFICADA
2,210061103945,1,1,1,1,502.0,498.9,475.6,363.2,700.0,C,B,507.94,Tem internet em casa,"De R$ 1.100,01 até R$ 1.650,00.",Baixa renda
3,210060214087,1,1,1,1,459.0,508.5,507.2,466.7,880.0,C,B,564.28,Tem internet em casa,"De R$ 1.100,01 até R$ 1.650,00.",Baixa renda
4,210059980948,1,1,1,1,402.5,379.2,446.9,338.3,560.0,B,A,425.38,Não tem internet em casa,"Até R$ 1.100,00",Baixa renda
9,210060801601,1,1,1,1,564.7,630.3,610.4,680.2,600.0,F,B,617.12,Tem internet em casa,"De R$ 2.750,01 até R$ 3.300,00.",Média renda
10,210059085130,1,1,1,1,644.9,620.2,626.9,736.3,860.0,B,B,697.66,Tem internet em casa,"Até R$ 1.100,00",Baixa renda


In [20]:
cols_export = [
    'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO', 'NU_NOTA_MEDIA',
    'INTERNET', 'RENDA_FAMILIAR', 'RENDA_SIMPLIFICADA'
]

df_export = df[cols_export].copy()

display(df_export)

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,NU_NOTA_MEDIA,INTERNET,RENDA_FAMILIAR,RENDA_SIMPLIFICADA
2,502.0,498.9,475.6,363.2,700.0,507.94,Tem internet em casa,"De R$ 1.100,01 até R$ 1.650,00.",Baixa renda
3,459.0,508.5,507.2,466.7,880.0,564.28,Tem internet em casa,"De R$ 1.100,01 até R$ 1.650,00.",Baixa renda
4,402.5,379.2,446.9,338.3,560.0,425.38,Não tem internet em casa,"Até R$ 1.100,00",Baixa renda
9,564.7,630.3,610.4,680.2,600.0,617.12,Tem internet em casa,"De R$ 2.750,01 até R$ 3.300,00.",Média renda
10,644.9,620.2,626.9,736.3,860.0,697.66,Tem internet em casa,"Até R$ 1.100,00",Baixa renda
...,...,...,...,...,...,...,...,...,...
3933946,568.6,605.2,598.2,496.8,500.0,553.76,Tem internet em casa,"De R$ 1.650,01 até R$ 2.200,00.",Baixa renda
3933948,476.2,542.9,545.2,530.5,600.0,538.96,Tem internet em casa,"De R$ 2.750,01 até R$ 3.300,00.",Média renda
3933950,566.2,605.6,613.7,547.3,540.0,574.56,Tem internet em casa,"De R$ 2.750,01 até R$ 3.300,00.",Média renda
3933951,377.2,535.6,610.6,644.4,640.0,561.56,Tem internet em casa,"De R$ 2.750,01 até R$ 3.300,00.",Média renda


In [ ]:
import os

# Nome do arquivo de saída
output_file = '../data/output/enem_2023_tratado.csv'

# Garante que o diretório de saída exista
output_dir = os.path.dirname(output_file)

if not os.path.exists(output_dir):
    os.makedirs(output_dir, exist_ok=True)

# Exporta o DataFrame para CSV
df_export.to_csv(output_file, index=False)

In [23]:
# Carregar o arquivo exportado para verificar
df_check = pd.read_csv(output_file)
print("\nVerificação do arquivo exportado:")
print(f"Shape: {df_check.shape}")
print("\nAmostra dos dados:")
display(df_check.sample(5))

print("\nEstatísticas descritivas:")
display(df_check.describe())

print("\nContagem de valores categóricos:")
print(df_check['INTERNET'].value_counts())
print(df_check['RENDA_SIMPLIFICADA'].value_counts())


Verificação do arquivo exportado:
Shape: (2678264, 9)

Amostra dos dados:


,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,NU_NOTA_MEDIA,INTERNET,RENDA_FAMILIAR,RENDA_SIMPLIFICADA
2329734,479.2,399.4,354.9,443.6,680.0,471.42,Tem internet em casa,"De R$ 1.100,01 até R$ 1.650,00.",Baixa renda
2572732,554.6,406.7,478.7,432.3,380.0,450.46,Tem internet em casa,"De R$ 3.300,01 até R$ 4.400,00.",Média renda
1414894,562.9,508.4,534.7,503.0,600.0,541.80,Tem internet em casa,"Até R$ 1.100,00",Baixa renda
956151,473.8,400.6,416.7,397.7,720.0,481.76,Não tem internet em casa,"De R$ 1.650,01 até R$ 2.200,00.",Baixa renda
1262008,441.1,405.0,381.5,336.1,640.0,440.74,Tem internet em casa,"Até R$ 1.100,00",Baixa renda



Estatísticas descritivas:


,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,NU_NOTA_MEDIA
count,2.678264e+06,2.678264e+06,2.678264e+06,2.678264e+06,2.678264e+06,2.678264e+06
mean,4.959129e+02,5.262042e+02,5.202973e+02,5.340268e+02,6.266020e+02,5.406086e+02
std,8.777803e+01,8.680629e+01,7.412317e+01,1.315292e+02,2.090321e+02,9.525702e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.406000e+02,4.712000e+02,4.737000e+02,4.314000e+02,5.200000e+02,4.756600e+02
50%,4.940000e+02,5.329000e+02,5.249000e+02,5.238000e+02,6.200000e+02,5.391800e+02
75%,5.513000e+02,5.869000e+02,5.718000e+02,6.303000e+02,7.800000e+02,6.063200e+02
max,8.684000e+02,8.230000e+02,8.208000e+02,9.586000e+02,1.000000e+03,8.625800e+02



Contagem de valores categóricos:
INTERNET
Tem internet em casa        2455844
Não tem internet em casa     222420
Name: count, dtype: int64
RENDA_SIMPLIFICADA
Baixa renda    1645031
Média renda     648868
Alta renda      384365
Name: count, dtype: int64


In [ ]:
# Gráfico 1: Comparação geral de notas por acesso à internet
fig1 = px.box(df, x='INTERNET', y='NU_NOTA_MEDIA', 
              title='Distribuição das Notas Médias por Acesso à Internet em Casa',
              labels={'NU_NOTA_MEDIA': 'Nota Média', 'INTERNET': 'Acesso à Internet em Casa'},
              color='INTERNET')
fig1.show()

In [ ]:
# Gráfico 2: Notas por área do conhecimento
areas = ['Ciências da Natureza', 'Ciências Humanas', 'Linguagens e Códigos', 'Matemática', 'Redação']
notas_cols = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']

fig2 = make_subplots(rows=2, cols=3, subplot_titles=areas)

for i, (area, nota) in enumerate(zip(areas, notas_cols)):
    row = (i // 3) + 1
    col = (i % 3) + 1
    
    for internet_type in df['INTERNET'].unique():
        subset = df[df['INTERNET'] == internet_type]
        fig2.add_trace(
            go.Box(y=subset[nota], name=internet_type, showlegend=(i==0)),
            row=row, col=col
        )

fig2.update_layout(title_text='Notas por Área do Conhecimento e Acesso à Internet', height=800)
fig2.show()

In [ ]:
# Gráfico 3: Interação entre renda e acesso à internet
# Vamos agrupar as rendas em categorias mais amplas para simplificar
def simplify_income(income):
    if income in ['A', 'B', 'C', 'D']:
        return 'Baixa renda'
    elif income in ['E', 'F', 'G', 'H']:
        return 'Média renda'
    else:
        return 'Alta renda'

df['RENDA_SIMPLIFICADA'] = df['Q006'].apply(simplify_income)

fig3 = px.box(df, x='RENDA_SIMPLIFICADA', y='NU_NOTA_MEDIA', color='INTERNET',
             facet_col='INTERNET',
             title='Nota Média por Nível de Renda e Acesso à Internet',
             labels={'NU_NOTA_MEDIA': 'Nota Média', 'RENDA_SIMPLIFICADA': 'Nível de Renda'})
fig3.show()

In [ ]:
# Calcular médias por grupo
media_com_internet = df[df['Q025'] == 'A']['NU_NOTA_MEDIA'].mean()
media_sem_internet = df[df['Q025'] == 'B']['NU_NOTA_MEDIA'].mean()
diferenca_absoluta = media_com_internet - media_sem_internet
diferenca_percentual = (diferenca_absoluta / media_sem_internet) * 100

print(f"Média com internet: {media_com_internet:.1f}")
print(f"Média sem internet: {media_sem_internet:.1f}")
print(f"Diferença absoluta: {diferenca_absoluta:.1f} pontos")
print(f"Diferença percentual: {diferenca_percentual:.1f}%")

# Calcular médias por renda e acesso à internet
tabela_media = df.groupby(['RENDA_SIMPLIFICADA', 'INTERNET'])['NU_NOTA_MEDIA'].mean().unstack()
print("\nMédias por renda e acesso à internet:")
print(tabela_media)